In [2]:
!pip install psycopg2-binary 
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 7.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 10.8 MB/s eta 0:00:00


In [138]:
from query_API import *

import pandas as pd
import dask.dataframe as dd

In [284]:
history_df = query_from_db("Select roundid, price, updatedat, pair, decimals From chainlink_history")
real_time_df = query_from_db("Select roundid, price, updatedat, pair, decimals From chainlink_real_time")

In [285]:
history_df.head()

,roundid,price,updatedat,pair,decimals
0,92233720368547771281,4652109401959,2021-12-31 22:13:16,BTC / USD,8
1,92233720368547771282,4639357085285,2021-12-31 22:55:11,BTC / USD,8
2,92233720368547771283,4635816164458,2021-12-31 23:56:36,BTC / USD,8
3,92233720368547771284,4661544706500,2022-01-01 00:42:51,BTC / USD,8
4,92233720368547771285,4670395911240,2022-01-01 00:56:01,BTC / USD,8


In [286]:
history_ddf = dd.from_pandas(history_df, npartitions=10)
real_time_ddf = dd.from_pandas(real_time_df, npartitions=10)

In [287]:
def percentage_diff(partition):
    return (partition.shift()-partition) / partition * 100

In [288]:
# calculate the percentage difference using the map_partitions() method
history_ddf['percentage_diff'] = history_ddf['price'].map_partitions(percentage_diff, meta=('x', 'f8'))

# drop the first row, which will have NaN values
history_ddf = history_ddf.dropna()

# convert object to float and round to the second decimal points
history_ddf['percentage_diff'] = history_ddf['percentage_diff'].astype(float).round(2)

# save dask dataframe to pandas dataframe
history_df = history_ddf.compute()

In [289]:
history_df.head()

,roundid,price,updatedat,pair,decimals,percentage_diff
1,92233720368547771282,4639357085285,2021-12-31 22:55:11,BTC / USD,8,0.27
2,92233720368547771283,4635816164458,2021-12-31 23:56:36,BTC / USD,8,0.08
3,92233720368547771284,4661544706500,2022-01-01 00:42:51,BTC / USD,8,-0.55
4,92233720368547771285,4670395911240,2022-01-01 00:56:01,BTC / USD,8,-0.19
5,92233720368547771286,4669785530693,2022-01-01 01:07:03,BTC / USD,8,0.01


In [187]:
from datetime import datetime
from datetime import timedelta
today = datetime.today()
yesterday = today - timedelta(days = 1)
the_day_before_yesterday = today - timedelta(days = 2)

In [188]:
today = today.strftime('%Y-%m-%d')
yesterday = yesterday.strftime('%Y-%m-%d')
the_day_before_yesterday = the_day_before_yesterday.strftime('%Y-%m-%d')

In [189]:
the_day_before_yesterday

'2023-04-23'

In [184]:
query = "SELECT roundid, price, updatedat, pair, decimals FROM chainlink_real_time WHERE updatedat >'{}'"
formatted_query = query.format(today)

In [174]:
formatted_query

"SELECT roundid, price, updatedat, pair, decimals FROM chainlink_real_time WHERE updatedat >'2023-04-25'"

In [175]:
real_time_df_test = query_from_db(formatted_query)

In [176]:
real_time_df_test

,roundid,price,updatedat,pair,decimals
0,92233720368547792782,2747456000000,2023-04-25 00:55:23,BTC / USD,8
1,92233720368547792783,2742257739000,2023-04-25 01:55:35,BTC / USD,8
2,92233720368547792784,2738978534871,2023-04-25 02:55:35,BTC / USD,8
3,92233720368547792785,2734687000000,2023-04-25 03:55:35,BTC / USD,8
4,92233720368547792786,2743571860000,2023-04-25 04:55:35,BTC / USD,8
5,92233720368547792787,2744891250000,2023-04-25 05:55:35,BTC / USD,8
6,92233720368547792788,2739069400000,2023-04-25 06:55:47,BTC / USD,8
7,92233720368547792789,2727647000000,2023-04-25 07:56:47,BTC / USD,8
8,92233720368547792790,2727676930000,2023-04-25 07:57:11,BTC / USD,8
9,92233720368547792791,2726921350000,2023-04-25 08:00:47,BTC / USD,8
